In [1]:
import gymnasium as gym
from stable_baselines3 import PPO
import tqdm
import time
import torch

In [2]:
# model = PPO("MlpPolicy", "CartPole-v1").learn(10_000)
env = gym.make("Pong-v4", render_mode="human")



# Test environment reset and step without LLM and rendering
obs, info = env.reset()

for step in range(100):  # Small loop for testing
    # Take a random action just to test environment stability
    action = env.action_space.sample()  
    obs, reward, done, truncated, info = env.step(action)
    
    if done or truncated:
        obs, info = env.reset()

env.close()

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]
2024-09-23 18:09:40.668 Python[1188:13595] +[IMKClient subclass]: chose IMKClient_Legacy
2024-09-23 18:09:40.668 Python[1188:13595] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


In [3]:

# # Create the Pong environment

# # Train a PPO model (You can adjust timesteps based on your computational resources)
# model = PPO("CnnPolicy", env, verbose=1)
# model.learn(total_timesteps=100000)  # Train for 100,000 steps

# # Save the model
# model.save("ppo_pong")


In [3]:
# Load the saved PPO model
model = PPO.load("ppo_pong")


In [4]:
from transformers import pipeline
import torch

# Check if GPU is available and set the device accordingly
device = 0 if torch.cuda.is_available() else -1

# Initialize the non-generative model (distilbert)
llm = pipeline("fill-mask", model="distilbert-base-uncased", device=device)

# Simple function to describe the game state
def describe_game_state(obs):
    return "The ball is moving towards the left. Your paddle is in the middle."

# Test the LLM alone to check its response (outside the loop)
def test_llm():
    prompt = describe_game_state(None).replace("left", "[MASK]")
    try:
        response = llm(prompt)[0]['sequence']
        print("LLM Response:", response)
    except Exception as e:
        print(f"LLM Error: {e}")

# Run a test of the LLM outside the loop to check its behavior
test_llm()

/Users/kpradjinata/Documents/Research/OpenAIGym/LLM/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/kpradjinata/Documents/Research/OpenAIGym/LLM/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


LLM Response: the ball is moving towards the net. your paddle is in the middle.


In [5]:
import time
from transformers import pipeline

# Initialize the LLM (distilgpt2) and set it to use the CPU
llm = pipeline("text-generation", model="distilgpt2")

# Simulate the Pong environment (you would replace this with actual environment data)
def get_observation():
    # Simulated game observation: ball and paddle positions
    return {'ball_x': 50, 'ball_y': 30, 'paddle_left_y': 25, 'paddle_right_y': 60}


def extract_features_from_obs(obs):
    # Convert the observation (which is likely a NumPy array) into an image format
    frame = np.array(obs, dtype=np.uint8)

    # Use OpenCV to process the image
    # Convert the frame to grayscale (if you want to detect based on intensity)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)

    # Use image processing techniques to detect the ball and paddle
    # For example, use thresholding or edge detection to find objects
    # You can modify these thresholds based on your game
    _, thresh = cv2.threshold(gray_frame, 127, 255, cv2.THRESH_BINARY)
    
    # Example: Detect the ball based on its color (e.g., white ball)
    # You would adjust the threshold depending on the color/shape of the ball
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Assume the largest contour is the ball
    ball_contour = max(contours, key=cv2.contourArea)
    ball_x, ball_y, ball_w, ball_h = cv2.boundingRect(ball_contour)

    # Example: Detect the paddle using a similar method
    # This is just a placeholder; adjust the logic based on your actual game
    paddle_contour = min(contours, key=cv2.contourArea)  # For example, smallest object
    paddle_x, paddle_y, paddle_w, paddle_h = cv2.boundingRect(paddle_contour)

    # Return the positions of the ball and paddle
    return ball_x, ball_y, 1, 1, paddle_y

# Function to describe the game state
def describe_game_state(obs):
    ball_x, ball_y, ball_vel_x, ball_vel_y, paddle_y = extract_features_from_obs(obs)
    
    if ball_y > paddle_y:
        relative_position = "above"
    elif ball_y < paddle_y:
        relative_position = "below"
    else:
        relative_position = "aligned with"
    
    # Log ball and paddle positions and velocities for debugging
    print(f"Ball: (x={ball_x}, y={ball_y}, vel_x={ball_vel_x}, vel_y={ball_vel_y}), Paddle: y={paddle_y}")
    
    description = (f"The ball is at position (x={ball_x}, y={ball_y}) moving at velocity "
                   f"(vx={ball_vel_x}, vy={ball_vel_y}). Your paddle is at y={paddle_y}. "
                   f"The ball is {relative_position} the paddle.")
    
    return description, ball_vel_y  # Pass ball_vel_y for decision-making

def get_llm_advice(game_state_description, ball_vel_y):
    # If the ball isn't moving vertically, stay still
    if ball_vel_y == 0:
        return "stay still"
    
    # Improved heuristic based on the ball's relative position and velocity
    if "above" in game_state_description and ball_vel_y > 0:
        return "move up"
    elif "below" in game_state_description and ball_vel_y < 0:
        return "move down"
    return "stay still"

    
# Function to map the LLM's advice to a game action
def llm_to_action(advice, obs):
    ball_x, ball_y, ball_vel_x, ball_vel_y, paddle_y = extract_features_from_obs(obs)
    
    if "move up" in advice and ball_y > paddle_y and ball_vel_y > 0:
        return 2  # Move paddle up
    elif "move down" in advice and ball_y < paddle_y and ball_vel_y < 0:
        return 0  # Move paddle down
    return 1  # Stay still (no action)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [6]:
import time
import tqdm
import cv2
import numpy as np

# Global variables to track ball's position and simulate velocity
previous_ball_x = None
previous_ball_y = None
ball_vel_x = 0
ball_vel_y = 0

def extract_features_from_obs(obs):
    global previous_ball_x, previous_ball_y, ball_vel_x, ball_vel_y
    
    # Convert the observation (which is likely a NumPy array) into an image format
    frame = np.array(obs, dtype=np.uint8)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)

    _, thresh = cv2.threshold(gray_frame, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Log the number of contours detected
    # print(f"Number of contours detected: {len(contours)}")
    
    # Check if contours were found
    if len(contours) > 0:
        # Attempt to find the ball based on contour size (ball should be small)
        ball_contour = None
        for contour in contours:
            area = cv2.contourArea(contour)
            # print(f"Contour Area: {area}")  # Log the area of each contour

            # Define a threshold to determine if it's the ball (adjust as needed)
            if 50 < area < 500:  # Adjust these values based on ball size in the game
                ball_contour = contour
                # print(f"Ball detected with area {area}.")  # Log ball detection
                break
        
        if ball_contour is not None:
            ball_x, ball_y, ball_w, ball_h = cv2.boundingRect(ball_contour)

            # Check if the ball's position is not changing and simulate movement
            if ball_y == previous_ball_y:  # If ball position hasn't changed, simulate movement
                ball_y += 5  # Simulate movement by incrementing the y-position
                print(f"Ball position was static, simulating movement: new y={ball_y}")
            
            # Calculate velocity based on the new or detected ball position
            if previous_ball_x is not None and previous_ball_y is not None:
                ball_vel_x = ball_x - previous_ball_x
                ball_vel_y = ball_y - previous_ball_y
            else:
                ball_vel_x = 0
                ball_vel_y = 0

            # Update previous position for future velocity calculation
            previous_ball_x, previous_ball_y = ball_x, ball_y

            # Predict future ball position
            predicted_ball_y = ball_y + ball_vel_y

            return ball_x, ball_y, predicted_ball_y, ball_vel_x, ball_vel_y, ball_y
        
        else:
            print("Ball not detected. Simulating movement.")
            return simulate_ball_movement()
    else:
        print("No contours detected. Simulating movement.")
        return simulate_ball_movement()

def simulate_ball_movement():
    """Simulate ball movement when it's not detected."""
    global previous_ball_y, ball_vel_y

    if previous_ball_y is None:
        previous_ball_y = 100  # Start in the middle of the screen

    # Simulate constant upward movement
    ball_vel_y = 2
    predicted_ball_y = previous_ball_y + ball_vel_y

    # Update previous position for next frame
    previous_ball_y = predicted_ball_y

    return 0, previous_ball_y, predicted_ball_y, 0, ball_vel_y, previous_ball_y

def describe_game_state(obs):
    # Extract features from the observation
    ball_x, ball_y, predicted_ball_y, ball_vel_x, ball_vel_y, paddle_y = extract_features_from_obs(obs)

    # Determine the relative position of the ball compared to the paddle
    if predicted_ball_y > paddle_y:
        relative_position = "above"
    elif predicted_ball_y < paddle_y:
        relative_position = "below"
    else:
        relative_position = "aligned with"
    
    # Log the game state for debugging
    # print(f"Ball: (x={ball_x}, y={ball_y}, predicted_y={predicted_ball_y}, vel_x={ball_vel_x}, vel_y={ball_vel_y}), Paddle: y={paddle_y}")
    
    # Generate a description of the game state
    description = (f"The ball is at position (x={ball_x}, y={ball_y}) moving at velocity "
                   f"(vx={ball_vel_x}, vy={ball_vel_y}). Your paddle is at y={paddle_y}. "
                   f"The ball is {relative_position} the paddle.")
    
    return description, ball_vel_y

def llm_to_action(advice, obs):
    # Extract features again (can be optimized by reusing previously computed values)
    ball_x, ball_y, predicted_ball_y, ball_vel_x, ball_vel_y, paddle_y = extract_features_from_obs(obs)
    
    # print(f"LLM Advice: {advice}")  # Print the LLM advice for debugging
    
    # Map the advice to actions (2 = move up, 0 = move down, 1 = stay still)
    if "move up" in advice and predicted_ball_y > paddle_y:
        return 2  # Move paddle up
    elif "move down" in advice and predicted_ball_y < paddle_y:
        return 0  # Move paddle down
    return 1  # Stay still

# Function to provide LLM advice based on ball position and velocity
def get_llm_advice(game_state_description, ball_vel_y):
    # Heuristic for advice based on the ball's relative position and velocity
    if "above" in game_state_description:
        return "move up"
    elif "below" in game_state_description:
        return "move down"
    return "stay still"

# Main game loop
obs, info = env.reset()

for step in tqdm.tqdm(range(100)):
    # Describe the game state for the LLM
    game_state_description, ball_vel_y = describe_game_state(obs)
    
    # Get advice from the LLM, passing ball_vel_y for smarter decisions
    advice = get_llm_advice(game_state_description, ball_vel_y)
    
    # Convert the LLM advice into an action the game can use
    rl_action = llm_to_action(advice, obs)
    
    # Step the environment using the chosen action
    obs, reward, done, truncated, info = env.step(rl_action)
    
    if reward != 0:
        print(f"Reward: {reward}")
    
    if step % 10 == 0:
        env.render()

    time.sleep(0.05)

    if done or truncated:
        obs, info = env.reset()

env.close()


  0%|                                                           | 0/100 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/kpradjinata/Documents/Research/OpenAIGym/LLM/venv/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:364: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(
  2%|█                                                  | 2/100 [00:00<00:09,  9.95it/s]

Ball position was static, simulating movement: new y=6
Ball position was static, simulating movement: new y=6
Ball position was static, simulating movement: new y=6


  5%|██▌                                                | 5/100 [00:00<00:10,  9.12it/s]

Ball position was static, simulating movement: new y=6
Ball position was static, simulating movement: new y=6
Ball position was static, simulating movement: new y=6


  7%|███▌                                               | 7/100 [00:00<00:11,  8.40it/s]

...

Ball position was static, simulating movement: new y=6
Ball position was static, simulating movement: new y=6
Ball position was static, simulating movement: new y=6


 97%|████████████████████████████████████████████████▌ | 97/100 [00:11<00:00,  9.36it/s]

Ball position was static, simulating movement: new y=6
Ball position was static, simulating movement: new y=6
Ball position was static, simulating movement: new y=6


100%|█████████████████████████████████████████████████| 100/100 [00:12<00:00,  7.73it/s]

Ball position was static, simulating movement: new y=6


In [27]:
obs, info = env.reset()

for step in tqdm.tqdm(range(100)):
    # print(f"Step {step}: Raw obs data (portion): {obs[0:5, 0:5]}")
    
    if previous_obs is not None:
        diff = np.sum(np.abs(obs - previous_obs))
        print(f"Step {step}: Difference between frames: {diff}")
        if diff == 0:
            print(f"Warning: No difference in frames between steps {step-1} and {step}. Resetting environment.")
            # Force a full environment reset to check if it gets unstuck
            obs, info = env.reset()  # Full reset of the environment after each step
    
    previous_obs = np.copy(obs)

    # Proceed with the rest of the game logic
    game_state_description, ball_vel_y = describe_game_state(obs)
    advice = get_llm_advice(game_state_description, ball_vel_y)
    rl_action = llm_to_action(advice, obs)
    
    # Step the environment with the chosen action
    obs, reward, done, truncated, info = env.step(rl_action)
    
    if diff == 0:
        print(f"Simulating frame change for step {step}.")
        obs += np.random.randint(-5, 5, size=obs.shape)  # Simulate changes in obs

    if reward != 0:
        print(f"Reward: {reward}")
    
    if step % 10 == 0:
        env.render()  # Disable this temporarily if needed

    time.sleep(0.05)

    if done or truncated:
        obs, info = env.reset()

env.close()


  2%|▉                                                | 2/100 [00:00<00:06, 15.18it/s]

Step 0: Difference between frames: 9600968
Ball position was static, simulating movement: new y=6
Step 1: Difference between frames: 16197688
Ball position was static, simulating movement: new y=6
Step 2: Difference between frames: 0
Ball position was static, simulating movement: new y=6
Simulating frame change for step 2.


UFuncTypeError: Cannot cast ufunc 'add' output from dtype('int64') to dtype('uint8') with casting rule 'same_kind'